<a href="https://colab.research.google.com/github/jongbokhi/Kaggle_Competition/blob/main/Regression_with_a_Mohs_Hardness_final_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📖 Ensemble algorithm(SuperLearner) +  CTGAN

## 📝 What I`ve done for improvinng performance on  "Regression with a Mohs Hardness" Competition.


✅ 1. Feed Neural Network with predictions of Superlearner(as Meta model)

I've been consistently working on enhancing the Superlearner, an Ensemble algorithm. In my approach, I opted for a Neural Network model as the Meta model instead of the linear regressor mentioned in the paper. Furthermore, the results demonstrated that the Neural Network outperformed the Linear model in terms of performance.

However, despite tuning hyperparameters, the effectiveness was limited, and my score remained stagnant between 0.49 and 0.43. During this period, I came across a discussion by @FabienPv and implemented the code shared by @Laurent Pourchot. This information proved incredibly valuable, and I'm grateful for it. I hadn't initially considered approaching the problem from a classification perspective.


---






✅ 2. Set labels from @Laurent Pourchot as a categorical feature



---




✅ 3. Generate synthetic test dataset using CTGAN

However, my determination to persist with the Superlearner model led me to set a goal of enhancing its performance. Motivated by the results obtained from @Laurent Pourchot's code, I decided to incorporate this information by labeling the test set. Subsequently, I utilized this labeled test set in conjunction with CTGAN to generate a synthetic test dataset, which I then combined with the original training set for further model training. After training CTGAN, I successfully generated 7000 rows in the synthetic test set, demonstrating a notably high quality with an average similarity of 92%.


---



📍 Result

I conducted a comparison between the two final prediction results generated by my model.

1. Averaged Prediction

 - 📌 Test score: 0.31027

2. Non-averaged predictions

 - 📌 Test score: 0.37055






In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.metrics import median_absolute_error, mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import tensorflow_probability as tfp

from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout

In [230]:
train = pd.read_csv('/content/drive/MyDrive/kaggle_Mohs hardness prediction/train.csv')
syn_test = pd.read_csv('/content/drive/MyDrive/kaggle_Mohs hardness prediction/sytheticdata_test2.csv')
train = train.drop("id",axis=1)

In [231]:
#combine synthetic data(Test set) with original training data

train = pd.concat([train, syn_test], axis=0, ignore_index=False)

In [232]:
############# I got an idea from @FabienPv
############# credited with the following code by @Laurent Pourchot

target_values = [1.75,2.55, 3.75,4.75,5.75,6.55,7.75,8.75,9.75]

def process_target(data):
    new_target = []
    for i in range(data.shape[0]):
        min_diff = 100
        min_j = 0
        for j in range(len(target_values)):
            diff = np.absolute(target_values[j]- data[i])
            if diff < min_diff :
                min_diff = diff
                min_j = j
        new_target.append(target_values[min_j])
    return pd.Series(new_target)

In [236]:
train["Hardness_class"] = process_target(train["Hardness"].values)

In [237]:
target_dict = dict(zip(np.unique(train['Hardness_class']),np.arange(0,9,1)))
target_undict = dict(zip(np.arange(0,9,1),np.unique(train['Hardness_class'])))

In [238]:
print(target_dict)
print(target_undict)

{1.75: 0, 2.55: 1, 3.75: 2, 4.75: 3, 5.75: 4, 6.55: 5, 7.75: 6, 8.75: 7, 9.75: 8}
{0: 1.75, 1: 2.55, 2: 3.75, 3: 4.75, 4: 5.75, 5: 6.55, 6: 7.75, 7: 8.75, 8: 9.75}


In [239]:
train['Hardness_class'] = train['Hardness_class'].map(target_dict)

In [240]:
train["Hardness_class"]=train["Hardness_class"].astype('category')

In [241]:
train["Hardness_class"]

0       4
1       5
2       1
3       4
4       4
       ..
6995    4
6996    3
6997    5
6998    2
6999    4
Name: Hardness_class, Length: 17407, dtype: category
Categories (9, int64): [0, 1, 2, 3, ..., 5, 6, 7, 8]

In [242]:
X = train.drop(['Hardness'],axis = 1).astype('float32')
y = train['Hardness'].astype('float32')

In [243]:
X.shape

(17407, 12)

In [244]:
features = list(X.columns)
print(features)

['allelectrons_Total', 'density_Total', 'allelectrons_Average', 'val_e_Average', 'atomicweight_Average', 'ionenergy_Average', 'el_neg_chi_Average', 'R_vdw_element_Average', 'R_cov_element_Average', 'zaratio_Average', 'density_Average', 'Hardness_class']


In [245]:
def loss_fn(y_true, y_pred):
    return tfp.stats.percentile(tf.abs(y_true - y_pred), q=50)

def metric_fn(y_true, y_pred):
    return tfp.stats.percentile(tf.abs(y_true - y_pred), q=100) - tfp.stats.percentile(tf.abs(y_true - y_pred), q=0), 4

In [246]:
def MedAE(y, yhat):
  return median_absolute_error(y, yhat)

In [247]:
SEED = 500
LR = 0.00001
SUBSAMPLE = 0.95

In [248]:
#############  credited with the following parameters from @FRANCESC OLIVERAS


cat_params ={'iterations' : 300,
            'depth' : 9,
            'verbose' : 0,
            'subsample' : SUBSAMPLE,
            'reg_lambda' : 0.075,
            'objective' : "MAE",
            'random_state' : SEED,
            'learning_rate' : LR,
            'min_child_samples' : 96,
            'colsample_bylevel' : 0.55
            }


gb_params ={ 'alpha':0.45,
            'max_depth':8,
            'subsample':SUBSAMPLE,
            'n_estimators':271,
            'min_samples_leaf':9,
            'learning_rate':LR,
            'random_state' : SEED,
            'min_samples_split':15,
            'loss' : "absolute_error"
            }


hgb_params ={'max_iter':251,
            'max_depth':10,
            'max_leaf_nodes':776,
            'learning_rate':LR,
            'random_state' : SEED,
            'min_samples_leaf': 16,
            'l2_regularization': 1.2,
            'loss' : "absolute_error"
            }


#############  credited with the following parameters from @IQMAN SINGH BHATIA

xgb_params = {'n_estimators': 1000, 'max_depth': 35, 'min_child_weight': 10.170221895950583,
              'learning_rate': 0.008383026974789366, 'subsample': 0.6705032929674755,
              'gamma': 0.6206804342071246, 'colsample_bytree': 0.6811067581297359,
              'colsample_bylevel': 0.8931207102930049, 'colsample_bynode': 0.9326927674256934}

lgbm_params = {'n_estimators': 1000, 'max_depth': 36, 'learning_rate': 0.01417471417676199,
               'min_child_weight': 1.1070027262610547, 'min_child_samples': 15,
               'subsample': 0.6061322281791591, 'subsample_freq': 3,
               'colsample_bytree': 0.4174456813911724, 'num_leaves': 55}

 # Build Superlearner model

In [249]:
### credited with the following code by @REMEK KINAS
def get_models():
    models = dict()

    #models['Cat'] = CatBoostRegressor(cat_features = cat_features, iterations=2500, verbose = 0)

    models['Xgb'] = XGBRegressor(**xgb_params, enable_categorical = True)

    models['LGBM'] = lgb.LGBMRegressor(**lgbm_params)

    #models['Gb'] = GradientBoostingRegressor(**gb_params)

    #models['SVR'] = SVR(gamma='scale')

    #models['KNN'] = KNeighborsRegressor(weights='distance')

    #models['Hgb'] = HistGradientBoostingRegressor(**hgb_params)

    #models['RF'] = RandomForestRegressor(n_estimators=30, criterion = 'absolute_error')

    return models

def get_out_of_fold_predictions(X, y, models):

    meta_X, meta_y = list(), list()

    kfold = KFold(n_splits=5, shuffle=True)

    print("* START - Out OF PREDICTION for base models * ")

    for fold_id, (train_ix, test_ix) in enumerate(kfold.split(X)):

        print(f'FOLD: {fold_id+1}')

        fold_yhats = list()

        train_X, test_X = X[train_ix], X[test_ix]

        train_y, test_y = y[train_ix], y[test_ix]

        meta_y.extend(test_y)

        for name, model in models.items():
            model.fit(train_X, train_y)

            yhat = model.predict(test_X)

            fold_yhats.append(yhat.reshape(len(yhat),1))

            score = loss_fn(test_y, yhat)

            print('     %s: %.3f' % (name, score))


        meta_X.append(np.hstack(fold_yhats))

    return np.vstack(meta_X), np.asarray(meta_y)




def fit_base_models(X, y, models):

    print("* START - Fitting base models * ")

    for name, model in models.items():

        print(f'   Fitting - {name} model')

        model.fit(X, y)

    print("* FINISH - Fitting base models * ")


earlystopping = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=70, verbose=2, mode="min" ,restore_best_weights=True),
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss", factor=0.8, patience=3, min_lr=0.00001),
terminatenan = tf.keras.callbacks.TerminateOnNaN()
callbacks_list = [earlystopping, reduce_lr, terminatenan]



def NN_tuned_model():
  NN_model = Sequential()

  # the Input layer and first hidden layer
  NN_model.add(Dense(16, input_shape=(2,), kernel_initializer='normal', activation='relu'))
  NN_model.add(BatchNormalization(epsilon=0.001))

  # the Second layer of the model
  NN_model.add(Dense(32, kernel_initializer='normal', activation='relu'))



  # the Third layer of the model
  NN_model.add(Dense(64, kernel_initializer='normal', activation='relu'))
  #NN_model.add(Dropout(0.2))

  # the Fourth layer of the model
  #NN_model.add(Dense(128, kernel_initializer='normal', activation='relu'))
  #NN_model.add(Dropout(0.2))



  # predicting a single number
  NN_model.add(Dense(1, kernel_initializer='normal'))


  # Compiling the model
  NN_model.compile(loss=loss_fn, optimizer=tf.keras.optimizers.Adam(0.013, beta_1=0.5), metrics=metric_fn, weighted_metrics=[])

  return NN_model



def fit_meta_model(X, y):

    print("* START - Fitting meta model * ")

    model = NN_tuned_model()


    model.fit(X, y, batch_size = 50, epochs = 500, callbacks = callbacks_list, validation_split=0.1, verbose=1)
    print("* FINISH - Fitting meta model * ")

    return model


def evaluate_models(X, y, models):

    for name, model in models.items():

        yhat = model.predict(X)

        score = loss_fn(y, yhat)

        print('%s: %.5f' % (model.__class__.__name__, score))



def super_learner_predictions(X, models, meta_model):

    meta_data_X = []

    for name, model in models.items():

        yhat = model.predict(X)

        meta_data_X.append(yhat.reshape(len(yhat),1))

    meta_data_X = np.hstack(meta_data_X)

    return meta_model.predict(meta_data_X)

In [250]:
# Train-Test Split

X_train, X_val, y_train, y_val = train_test_split(X, y,test_size=0.33, random_state = 4344)

In [251]:
#Get Base models
models = get_models()

In [252]:
# get out of fold predictions
meta_X, meta_y = get_out_of_fold_predictions(X_train[features].values, y_train.values, models)

* START - Out OF PREDICTION for base models * 
FOLD: 1
     Xgb: 0.296
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001026 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2808
[LightGBM] [Info] Number of data points in the train set: 9329, number of used features: 12
[LightGBM] [Info] Start training from score 4.631861
     LGBM: 0.282
FOLD: 2
     Xgb: 0.298
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000990 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2808
[LightGBM] [Info] Number of data points in the train set: 9329, number of used features: 12
[LightGBM] [Info] Start training from score 4.641417
     LGBM: 0.296
FOLD: 3
     Xgb: 0.283
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000976 seconds.
You can set `force_col_wise=true` to remove the overhead.
[Li

In [254]:
# fit base models

fit_base_models(X_train[features], y_train, models)

* START - Fitting base models * 
   Fitting - Xgb model
   Fitting - LGBM model
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001270 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2810
[LightGBM] [Info] Number of data points in the train set: 11662, number of used features: 12
[LightGBM] [Info] Start training from score 4.635857
* FINISH - Fitting base models * 


In [255]:
# fit the meta model

meta_model = fit_meta_model(meta_X, meta_y)

* START - Fitting meta model * 
Epoch 1/500
210/210 [==============================] - 2s 4ms/step - loss: 1.0274 - metric_fn: 3.7772 - val_loss: 1.4556 - val_metric_fn: 3.8776 - lr: 0.0130
Epoch 2/500
210/210 [==============================] - 1s 3ms/step - loss: 0.6762 - metric_fn: 3.5494 - val_loss: 0.4258 - val_metric_fn: 3.4034 - lr: 0.0130
Epoch 3/500
210/210 [==============================] - 1s 3ms/step - loss: 0.5545 - metric_fn: 3.5735 - val_loss: 0.6188 - val_metric_fn: 3.6197 - lr: 0.0130
Epoch 4/500
210/210 [==============================] - 1s 2ms/step - loss: 0.5788 - metric_fn: 3.5967 - val_loss: 0.7183 - val_metric_fn: 3.6199 - lr: 0.0130
Epoch 5/500
210/210 [==============================] - 1s 3ms/step - loss: 0.5235 - metric_fn: 3.5183 - val_loss: 0.9521 - val_metric_fn: 3.7398 - lr: 0.0130
Epoch 6/500
210/210 [==============================] - 1s 3ms/step - loss: 0.5072 - metric_fn: 3.5629 - val_loss: 0.4350 - val_metric_fn: 3.5303 - lr: 0.0130
Epoch 7/500
210/210 

In [256]:
# evaluate base models
evaluate_models(X_val[features], y_val, models)

XGBRegressor: 0.27649
LGBMRegressor: 0.27968


In [257]:
# evaluate meta model
y_hat = super_learner_predictions(X_val[features], models, meta_model)

score = MedAE(y_val, y_hat)

print('Super Learner: %.3f' % score)

180/180 [==============================] - 0s 1ms/step
Super Learner: 0.133


In [258]:
#Train Base models on full Data for Final Prediction

final_models = get_models()

meta_X_full, meta_y_full = get_out_of_fold_predictions(X[features].values, y.values, final_models)

fit_base_models(X[features].values, y.values, final_models)

* START - Out OF PREDICTION for base models * 
FOLD: 1
     Xgb: 0.274
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001498 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2810
[LightGBM] [Info] Number of data points in the train set: 13925, number of used features: 12
[LightGBM] [Info] Start training from score 4.628652
     LGBM: 0.276
FOLD: 2
     Xgb: 0.280
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001444 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2811
[LightGBM] [Info] Number of data points in the train set: 13925, number of used features: 12
[LightGBM] [Info] Start training from score 4.632350
     LGBM: 0.286
FOLD: 3
     Xgb: 0.275
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001427 seconds.
You can set `force_col_wise=true` to remove the overhead.
[

In [259]:
#Train Meta models on full Data for Final Prediction

meta_model_full = fit_meta_model(meta_X_full, meta_y_full)

* START - Fitting meta model * 
Epoch 1/500
314/314 [==============================] - 3s 3ms/step - loss: 0.9062 - metric_fn: 3.9123 - val_loss: 0.3997 - val_metric_fn: 3.5660 - lr: 0.0130
Epoch 2/500
314/314 [==============================] - 1s 3ms/step - loss: 0.5830 - metric_fn: 3.5079 - val_loss: 0.4074 - val_metric_fn: 3.5131 - lr: 0.0130
Epoch 3/500
314/314 [==============================] - 1s 2ms/step - loss: 0.5509 - metric_fn: 3.5094 - val_loss: 0.4457 - val_metric_fn: 3.5576 - lr: 0.0130
Epoch 4/500
314/314 [==============================] - 1s 3ms/step - loss: 0.4961 - metric_fn: 3.4389 - val_loss: 0.4258 - val_metric_fn: 3.5595 - lr: 0.0130
Epoch 5/500
314/314 [==============================] - 1s 3ms/step - loss: 0.4874 - metric_fn: 3.4976 - val_loss: 0.5256 - val_metric_fn: 3.4965 - lr: 0.0130
Epoch 6/500
314/314 [==============================] - 1s 3ms/step - loss: 0.4814 - metric_fn: 3.4788 - val_loss: 0.6920 - val_metric_fn: 3.6529 - lr: 0.0130
Epoch 7/500
314/314 

In [301]:
test_df = pd.read_csv('/content/drive/MyDrive/kaggle_Mohs hardness prediction/test.csv')

In [302]:
hardness_clf = pd.read_csv('/content/drive/MyDrive/kaggle_Mohs hardness prediction/hardness_class_pre.csv')

In [303]:
hardness_clf

,Hardness_pre
0,2.55
1,2.55
2,5.75
3,5.75
4,5.75
...,...
6934,5.75
6935,3.75
6936,5.75
6937,4.75


In [304]:
test_df = pd.concat([test_df, hardness_clf], axis=1, ignore_index=False)

In [305]:
test_df = test_df.rename(columns={'Hardness_pre': 'Hardness_class'})

In [306]:
print(test_df.columns)

Index(['id', 'allelectrons_Total', 'density_Total', 'allelectrons_Average',
       'val_e_Average', 'atomicweight_Average', 'ionenergy_Average',
       'el_neg_chi_Average', 'R_vdw_element_Average', 'R_cov_element_Average',
       'zaratio_Average', 'density_Average', 'Hardness_class'],
      dtype='object')


In [307]:
target_dict = dict(zip(np.unique(test_df['Hardness_class']),np.arange(0,9,1)))
target_undict = dict(zip(np.arange(0,9,1),np.unique(test_df['Hardness_class'])))
print(target_dict)
print(target_undict)

{1.75: 0, 2.55: 1, 3.75: 2, 4.75: 3, 5.75: 4, 6.55: 5, 7.75: 6, 8.75: 7, 9.75: 8}
{0: 1.75, 1: 2.55, 2: 3.75, 3: 4.75, 4: 5.75, 5: 6.55, 6: 7.75, 7: 8.75, 8: 9.75}


In [308]:
test_df['Hardness_class'] = test_df['Hardness_class'].map(target_dict)

In [309]:
test_df.head(5)

,id,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness_class
0,10407,884.0,121.420000,35.36,5.28,82.561240,9.370384,2.298,1.9072,1.200000,0.461899,1.79459,1
1,10408,90.0,9.931960,18.00,5.60,39.568056,12.086300,2.868,1.6520,0.864000,0.476196,1.41194,1
2,10409,116.0,7.767992,11.60,4.80,23.231818,11.023840,2.644,1.7940,0.960000,0.499514,0.78834,4
3,10410,100.0,9.107996,10.00,4.80,20.298893,12.086300,2.828,1.6620,0.792000,0.495796,1.20466,4
4,10411,55.0,4.030000,11.00,4.00,22.977675,11.280950,2.455,1.7500,0.893333,0.476095,0.93456,4


In [310]:
# Superlearner Final Prediction

test_df['Hardness'] = super_learner_predictions(test_df[features].values, final_models, meta_model_full)

217/217 [==============================] - 0s 2ms/step


In [311]:
test_df

,id,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness_class,Hardness
0,10407,884.0,121.420000,35.360000,5.280000,82.561240,9.370384,2.298000,1.907200,1.200000,0.461899,1.79459,1,2.630082
1,10408,90.0,9.931960,18.000000,5.600000,39.568056,12.086300,2.868000,1.652000,0.864000,0.476196,1.41194,1,2.630082
2,10409,116.0,7.767992,11.600000,4.800000,23.231818,11.023840,2.644000,1.794000,0.960000,0.499514,0.78834,4,5.870550
3,10410,100.0,9.107996,10.000000,4.800000,20.298893,12.086300,2.828000,1.662000,0.792000,0.495796,1.20466,4,5.870550
4,10411,55.0,4.030000,11.000000,4.000000,22.977675,11.280950,2.455000,1.750000,0.893333,0.476095,0.93456,4,5.870550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6934,17341,446.0,36.135392,9.636364,4.636364,19.689448,11.045515,2.768636,1.769697,0.871818,0.498702,0.11548,4,5.870550
6935,17342,36.0,3.550000,18.000000,4.000000,36.068500,8.236600,1.790000,2.055000,1.390000,0.499000,1.35045,2,2.630082
6936,17343,68.0,4.545328,11.333333,5.333333,22.688853,10.938358,2.876667,1.713333,0.916667,0.499074,0.73425,4,5.870550
6937,17344,40.0,2.334164,6.666667,4.000000,13.016128,12.700467,2.770000,1.476667,0.616667,0.663797,0.51227,3,5.870550


In [312]:
test_df['Hardness_class'] = test_df['Hardness_class'].map(target_undict)

In [313]:
test_df.head(5)

,id,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness_class,Hardness
0,10407,884.0,121.420000,35.36,5.28,82.561240,9.370384,2.298,1.9072,1.200000,0.461899,1.79459,2.55,2.630082
1,10408,90.0,9.931960,18.00,5.60,39.568056,12.086300,2.868,1.6520,0.864000,0.476196,1.41194,2.55,2.630082
2,10409,116.0,7.767992,11.60,4.80,23.231818,11.023840,2.644,1.7940,0.960000,0.499514,0.78834,5.75,5.870550
3,10410,100.0,9.107996,10.00,4.80,20.298893,12.086300,2.828,1.6620,0.792000,0.495796,1.20466,5.75,5.870550
4,10411,55.0,4.030000,11.00,4.00,22.977675,11.280950,2.455,1.7500,0.893333,0.476095,0.93456,5.75,5.870550


In [315]:
test_df['Hardness_class'] = test_df['Hardness_class'].astype('float32')

In [316]:
submission_df_normal = test_df[['id', 'Hardness']]

In [317]:
## test score: 0.37055

submission_df_normal.applymap(lambda x: '{:.2f}'.format(x))

,id,Hardness
0,10407.00,2.63
1,10408.00,2.63
2,10409.00,5.87
3,10410.00,5.87
4,10411.00,5.87
...,...,...
6934,17341.00,5.87
6935,17342.00,2.63
6936,17343.00,5.87
6937,17344.00,5.87


In [318]:
submission_df_normal.to_csv("/content/drive/MyDrive/kaggle_Mohs hardness prediction/submission_normal30.csv", index=False)

I tried to average ''Hardness_class' and 'Hardness'. It showed improved socres.

In [321]:
test_df['avg_pre']= ((test_df['Hardness_class'] + test_df['Hardness']) / 2)

In [322]:
test_df

,id,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness_class,Hardness,avg_pre
0,10407,884.0,121.420000,35.360000,5.280000,82.561240,9.370384,2.298000,1.907200,1.200000,0.461899,1.79459,2.55,2.630082,2.590041
1,10408,90.0,9.931960,18.000000,5.600000,39.568056,12.086300,2.868000,1.652000,0.864000,0.476196,1.41194,2.55,2.630082,2.590041
2,10409,116.0,7.767992,11.600000,4.800000,23.231818,11.023840,2.644000,1.794000,0.960000,0.499514,0.78834,5.75,5.870550,5.810275
3,10410,100.0,9.107996,10.000000,4.800000,20.298893,12.086300,2.828000,1.662000,0.792000,0.495796,1.20466,5.75,5.870550,5.810275
4,10411,55.0,4.030000,11.000000,4.000000,22.977675,11.280950,2.455000,1.750000,0.893333,0.476095,0.93456,5.75,5.870550,5.810275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6934,17341,446.0,36.135392,9.636364,4.636364,19.689448,11.045515,2.768636,1.769697,0.871818,0.498702,0.11548,5.75,5.870550,5.810275
6935,17342,36.0,3.550000,18.000000,4.000000,36.068500,8.236600,1.790000,2.055000,1.390000,0.499000,1.35045,3.75,2.630082,3.190041
6936,17343,68.0,4.545328,11.333333,5.333333,22.688853,10.938358,2.876667,1.713333,0.916667,0.499074,0.73425,5.75,5.870550,5.810275
6937,17344,40.0,2.334164,6.666667,4.000000,13.016128,12.700467,2.770000,1.476667,0.616667,0.663797,0.51227,4.75,5.870550,5.310275


In [323]:
submission_df_avg = test_df[['id', 'avg_pre']]

In [324]:
submission_df_avg.rename(columns={'avg_pre': 'Hardness'}, inplace=True)

<ipython-input-324-4a1bbb19cc88>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_df_avg.rename(columns={'avg_pre': 'Hardness'}, inplace=True)


In [325]:
## test score: 0.31027
submission_df_avg.applymap(lambda x: '{:.2f}'.format(x))

,id,Hardness
0,10407.00,2.59
1,10408.00,2.59
2,10409.00,5.81
3,10410.00,5.81
4,10411.00,5.81
...,...,...
6934,17341.00,5.81
6935,17342.00,3.19
6936,17343.00,5.81
6937,17344.00,5.31


In [326]:
submission_df_avg.to_csv("/content/drive/MyDrive/kaggle_Mohs hardness prediction/submission_avg30.csv", index=False)